In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import shapely.geometry as shpg
import matplotlib.pyplot as plt
import salem
from glob import glob
from matplotlib.backends.backend_pdf import PdfPages
from salem import datasets
from cleo import Map
import shapely.ops
%pylab inline

# Files

In [ ]:
wgms_dir = '..\\DOI-WGMS-FoG-2014-09'
rgi_dir_sup = '..\\rgi50'
output_dir = '..\\WGMS_LINKS'
f_A = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-A-GENERAL-INFORMATION.csv')
f_EE = os.path.join(wgms_dir, 'WGMS-FoG-2014-09-EE-MASS-BALANCE.csv')
r_rgi = os.path.join(rgi_dir_sup, '00_rgi50_regions\\00_rgi50_O1Regions.shp')
f_rgi_links = '..\\00_rgi50_links.csv'
f_gla = '..\\Glathida_2014\\T.csv'
f_lec = '..\\Leclercq\\glacier_properties.txt'

# in order not to treat the glaciers again that we have already linked
f_links = '..\\WGMS_LINKS\\Manual_links_WGMS_to_RGI_GlaThiDa_Leclercq_ALPS_20160221.csv'

# Haversine function

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between one point 
    on the earth and an array of points (specified in decimal degrees)
    """
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371000 # Radius of earth in meters
    return c * r

# Read RGI - only regions this time

In [ ]:
rgi_regions = gpd.read_file(r_rgi)

# Read also the GlaThiDa link file

In [ ]:
gla = pd.read_csv(f_gla, header=0, encoding='iso8859_15', sep=';', low_memory=False)
gla.head(1)

# Read also the Leclercq link file

In [ ]:
lec = pd.read_csv(f_lec, sep='\t', na_values='-99999')
lec = lec.rename(columns=lambda x: x.strip())
lec.head(1)

# Read also the file with already linked glaciers in the Alps

In [ ]:
safe = pd.read_csv(f_links)
safe_ids = safe.WGMS_ID.values

# Select WGMS glaciers with > 5 MB measurements

In [ ]:
pda = pd.read_csv(f_A, encoding='iso8859_15')
pdee = pd.read_csv(f_EE, encoding='iso8859_15')

In [ ]:
all_ids = pda.WGMS_ID.values
pdee = pdee[pdee.WGMS_ID.isin(all_ids)]
# Where we have yearly measures and no altitude range
pdee = pdee[pdee.LOWER_BOUND.isin([9999])]
gp_id = pdee.groupby('WGMS_ID')
ids_5 = []
for wgmsid, group in gp_id:
    if np.sum(np.isfinite(group.ANNUAL_BALANCE.values)) >= 5:
        ids_5.append(wgmsid)
print len(ids_5)

# make them th selected IDs
ids_sel = ids_5

# Exclude those glaciers that are already linked to RGI 

Check if rgi links are contained totally in the selected glaciers

In [ ]:
rgi_links = pd.read_csv(f_rgi_links, skiprows=[0,1])

not_cont_ids = set(rgi_links.FoGId.values) - set(ids_sel)
print len(not_cont_ids)
subset =  rgi_links[rgi_links.FoGId.isin(not_cont_ids)][['Name', 'FoGId', 'NYears']]
subset = subset[subset.NYears >= 5]
print subset

No, the RGI links are not totally contained in the selected glaciers, because MOST OF THOSE NOT CONTAINED have less than 5 MB measurements.
Exceptions are Langjoekull Southern Dome (13MB) and SU5A03003026 Obruchev Glacier (24MB)

In [ ]:
print pda[pda.WGMS_ID == 729]
for wgmsid, group in gp_id:
    if wgmsid == 729:
        print np.sum(np.isfinite(group.ANNUAL_BALANCE.values))
        
print pda[pda.WGMS_ID == 3101]
for wgmsid, group in gp_id:
    if wgmsid == 729:
        print np.sum(np.isfinite(group.ANNUAL_BALANCE.values))

Okay: IF the value for 'NYears' is the number of MB measurements, they simply do only have 4 measurements each in the pdee file. SO we still can do the following:

Subtract already linked RGI glaciers from selected

In [ ]:
ids_sel = set(ids_sel) - set(rgi_links.FoGId.values)
print len(ids_sel)

# Check for each of the RGI regions which WGMS glaciers are contained and produce plots, accordingly (not all RGI must be in memory at same time then)

## Assign first RGI region to WGMS glaciers

In [ ]:
ct = 0
pda_sel = pda[pda.WGMS_ID.isin(ids_sel)]
pda_sel['rgi_region'] = ''
print len(pda_sel)
for index, row in pda_sel.iterrows():
    for poly in rgi_regions.geometry.values:
        if shpg.Point(row['LONGITUDE'], row['LATITUDE']).within(poly):
            pda_sel.rgi_region[pda_sel.index == index] = rgi_regions[rgi_regions.geometry == poly].Primary_ID.iloc[0]
            ct+=1
            if ct % 10 ==0:
                print ct
            break
print 'done.'

In [ ]:
pda_sel.to_csv(os.path.join(output_dir, 'WGMS_RGI_regions_table.csv'), encoding='iso8859_15')
pda_sel.head(1)

## A dictionary that links name of the region to RGI file

In [ ]:
reg_dict = {'Alaska':'01_rgi50_Alaska\\01_rgi50_Alaska.shp',
            'Antarctic and Subantarctic':'19_rgi50_AntarcticSubantarctic\\19_rgi50_AntarcticSubantarctic.shp',
            'Arctic Canada North':'03_rgi50_ArcticCanadaNorth\\03_rgi50_ArcticCanadaNorth.shp', 
            'Arctic Canada South':'04_rgi50_ArcticCanadaSouth\\04_rgi50_ArcticCanadaSouth.shp',
            'Caucasus and Middle East':'12_rgi50_CaucasusMiddleEast\\12_rgi50_CaucasusMiddleEast.shp',
            'Central Asia':'13_rgi50_CentralAsia\\13_rgi50_CentralAsia.shp',
            'Central Europe':'11_rgi50_CentralEurope\\11_rgi50_CentralEurope.shp',
            'Greenland':'05_rgi50_GreenlandPeriphery\\05_rgi50_GreenlandPeriphery.shp',
            'Iceland':'06_rgi50_Iceland\\06_rgi50_Iceland.shp',
            'Low Latitudes':'16_rgi50_LowLatitudes\\16_rgi50_LowLatitudes.shp',
            'New Zealand':'18_rgi50_NewZealand\\18_rgi50_NewZealand.shp',
            'North Asia':'10_rgi50_NorthAsia\\10_rgi50_NorthAsia.shp',
            'Russian Arctic':'09_rgi50_RussianArctic\\09_rgi50_RussianArctic.shp',
            'Scandinavia':'08_rgi50_Scandinavia\\08_rgi50_Scandinavia.shp',
            'South Asia East':'15_rgi50_SouthAsiaEast\\15_rgi50_SouthAsiaEast.shp',
            'South Asia West':'14_rgi50_SouthAsiaWest\\14_rgi50_SouthAsiaWest.shp', 
            'Southern Andes':'17_rgi50_SouthernAndes\\17_rgi50_SouthernAndes.shp',
            'Svalbard':'07_rgi50_Svalbard\\07_rgi50_Svalbard.shp',
            'Western Canada and US':'02_rgi50_WesternCanadaUS\\02_rgi50_WesternCanadaUS.shp'}

# One manual extension necessary (glacier not in a single region polygon)

In [ ]:
pda_sel.rgi_region[pda_sel.NAME == 'HAMAGURI YUKI'] = 'North Asia'

# Make a pandas DataFrame for the result links - basically a copy of the pda frame

In [ ]:
results = pda_sel[['POLITICAL_UNIT', 'NAME', 'WGMS_ID', 'LATITUDE', 'LONGITUDE']]
results['RGI_ID'] = np.nan
results['GlaThiDa_ID'] = np.nan
results['Leclercq_ID'] = np.nan
results['remark'] = np.nan
lec.columns.values

In [ ]:
with PdfPages(os.path.join(output_dir, 'WGMS_Glaciers_WORLD_ALL.pdf' )) as pdf:
    for regio in np.unique(pda_sel.rgi_region.values):
        print regio
        pda_sel_regio = pda_sel[pda_sel.rgi_region == regio]

        pdrgi = gpd.read_file(os.path.join(rgi_dir_sup,reg_dict[regio]), encoding='iso8859_15')

        curr = 1  # current plot
        total = len(pda_sel_regio)  # total number of plots ()

        for gid in pda_sel_regio.WGMS_ID.values:
            glacier = pda_sel_regio[pda_sel_regio.WGMS_ID == gid].iloc[0]
            print 'Glacier: ', glacier.NAME, ' ', glacier.GEN_LOCATION, ' ', glacier.SPEC_LOCATION
            lon, lat = glacier.LONGITUDE, glacier.LATITUDE
            pdrgi['DIST'] = haversine(lon, lat, pdrgi.CenLon.values, pdrgi.CenLat.values)
            sortrgi = pdrgi.sort(columns='DIST')

            # For GoogleMap we need a lon lat range to generate the map
            mmlon = [lon, lon]
            mmlat = [lat, lat]

            ###################################################################################################
            # For the case the glacier is already checked manually (in the Alps)
            if regio == 'Central Europe' and (gid in safe_ids):
                rgi_id = safe.RGI_ID[safe.WGMS_ID == gid].values[0]
                lec_id = safe.Leclercq_ID[safe.WGMS_ID == gid].values[0]
                gla_id = safe.GlaThiDa_ID[safe.WGMS_ID == gid].values[0]

                # RGI x/y
                rgi_poly = pdrgi[pdrgi.RGIId == rgi_id].iloc[0]
                if rgi_poly.geometry.type == 'Polygon':
                    rgi_x, rgi_y = rgi_poly.geometry.exterior.xy
                elif rgi_poly.geometry.type == 'MultiPolygon':
                    # buffer is necessary as some multi-polygons are self-intersecting
                    allparts = [p.buffer(0) for p in rgi_poly.geometry] 
                    rgi_poly.geometry = shapely.ops.cascaded_union(allparts)
                    rgi_x, rgi_y = rgi_poly.geometry.exterior.xy

                # GlaThiDa x/y
                if not pd.isnull(gla_id):
                    print gla_id
                    gla_x, gla_y = gla[gla.GlaThiDa_ID == gla_id].LON.values[0], gla[gla.GlaThiDa_ID == gla_id].LAT.values[0]
                else:
                    gla_x, gla_y = np.nan, np.nan

                # Leclercq x/y
                if not pd.isnull(lec_id):
                    lec_x, lec_y = lec[lec.ID == lec_id].lon.values[0], lec[lec.ID == lec_id].lat.values[0]
                else:
                    lec_x, lec_y = np.nan, np.nan
                
                mmlon = [np.nanmin(np.append(np.append(np.append(mmlon, rgi_x), gla_x), lec_x)),
                         np.nanmax(np.append(np.append(np.append(mmlon, rgi_x), gla_x), lec_x))]
                mmlat = [np.nanmin(np.append(np.append(np.append(mmlat, rgi_y), gla_y), lec_y)),
                         np.nanmax(np.append(np.append(np.append(mmlat, rgi_y), gla_y), lec_y))]

                # Make a local map where to plot the polygons
                local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat) # also possible:  maptype='terrain'
                local_map = Map(local.grid, countries=False, nx=640)
                local_map.set_lonlat_countours()

                # Prepare the figure
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
                ax1.set_title('{}: '.format(gid) + glacier.POLITICAL_UNIT + '-' + glacier.NAME)

                # RGI polygon label
                if rgi_poly.Name == None or len(rgi_poly.Name)==1:
                    plabel =  str(rgi_poly.RGIId)+'\n'+str(rgi_poly.Area)+'km2'
                else:
                    plabel =  str(rgi_poly.RGIId)+'\n'+str(rgi_poly.Area)+'km2\n'+rgi_poly.Name.encode('ascii', 'ignore')

                # RGI centroid
                local_map.set_geometry(shpg.Point(rgi_poly.CenLon, rgi_poly.CenLat), edgecolor='k', marker='x', 
                                       linewidth=4, markersize=100, zorder=50, text='RGI', text_delta=(0.01, 0.0))
                local_map.set_geometry(rgi_poly.geometry.exterior, color=colors[i], linewidth=3, label=plabel)

                # Plot the WGMS point
                local_map.set_geometry(shpg.Point(lon, lat), color='g', marker='x', linewidth=4, markersize=100, 
                                       zorder=50, text='WGMS', text_delta=(0.01, 0.01))

                # Plot the GlaThiDa point
                if not pd.isnull(gla_id):
                    local_map.set_geometry(shpg.Point(gla_x, gla_y), color='g', marker='x', linewidth=4, markersize=100, 
                                           zorder=50, text='GLA', text_delta=(0.01, -0.01))
                
                # Plot the Leclercq point
                if not pd.isnull(lec_id):
                    local_map.set_geometry(shpg.Point(lec_x, lec_y), color='g', marker='x', linewidth=4, markersize=100, 
                                           zorder=50, text='LEC', text_delta=(0.01, -0.02))

                local_map.set_rgb(local.get_vardata())
                local_map.visualize(ax=ax1, addcbar=False)

                local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat, maptype='terrain')
                local_map.set_rgb(local.get_vardata())
                local_map.visualize(ax=ax2, addcbar=False)
                plt.subplots_adjust(left=0.04, right=0.80, top=0.94, bottom=0.07)
                handles, labels = ax1.get_legend_handles_labels()
                dummyline = plt.Line2D((0,1),(0,0), color='w', linewidth=0, linestyle='') # for the 'ALREADY CHECKED' entry
                plt.legend(handles+[dummyline], labels+['ALREADY CHECKED'],
                           bbox_to_anchor=(1.02, 1.), fontsize=18, loc=2, borderaxespad=0, frameon=False, numpoints=1,
                           scatterpoints=1)
                pdf.savefig(fig)
                plt.close()
                
                # write results
                results.GlaThiDa_ID[results.WGMS_ID == gid] = gla_id
                results.Leclercq_ID[results.WGMS_ID == gid] = lec_id
                results.RGI_ID[results.WGMS_ID == gid] = rgi_id
                results.remark[results.WGMS_ID == gid] = 'ok'
                continue
            ##########################################################################################################
            

            # for all other glaciers in the world
            for i in np.arange(0,5):
                rgig = sortrgi.iloc[i]
                # In case the glacier is a MultiPolygon we account for this here:
                if rgig.geometry.type == 'Polygon':
                    rgi_x, rgi_y = rgig.geometry.exterior.xy
                elif rgig.geometry.type == 'MultiPolygon':
                    # buffer is necessary as some multi-polygons are self-intersecting
                    #allparts = [p.buffer(0) for p in rgig.geometry] 
                    #rgig.geometry = shapely.ops.cascaded_union(allparts)
                    rgig.geometry = rgig.geometry.convex_hull
                    rgi_x, rgi_y = rgig.geometry.exterior.xy

                mmlon = [np.min(np.append(mmlon, rgi_x)), np.max(np.append(mmlon, rgi_x))]
                mmlat = [np.min(np.append(mmlat, rgi_y)), np.max(np.append(mmlat, rgi_y))]

            # Make a local map where to plot the polygons
            local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat) # also possible:  maptype='terrain'
            local_map = Map(local.grid, countries=False, nx=640)
            local_map.set_lonlat_countours()
            
            # find the closest GlaThiDa and Leclercq points like this
            gi, gj = local_map.grid.transform(gla.LON.values, gla.LAT.values, nearest=True, maskout=True)
            gin = np.where(gi.mask == False)[0] # 'GlaThiDa INside map'
            
            li, lj = local_map.grid.transform(lec.lon.values, lec.lat.values, nearest=True, maskout=True)
            lin = np.where(li.mask == False)[0] # 'Leclercq INside map

            # Prepare the figure
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
            ax1.set_title('{}: '.format(gid) + glacier.POLITICAL_UNIT.decode("ascii", "ignore") + '-' + 
                          glacier.NAME.decode("ascii", "ignore"))
            ax2.set_title("Smallest Haversine distance: %.2f m" % 
                          (haversine(lon, lat, sortrgi.iloc[0].CenLon, sortrgi.iloc[0].CenLat)))

            colors = ['red', 'orange', 'green', 'blue', 'purple', 'magenta']
            for i in np.arange(0,5):
                rgig = sortrgi.iloc[i]
                # In case the glacier is a MultiPolygon we (again) account for this here:
                if rgig.geometry.type == 'Polygon':
                    rgi_x, rgi_y = rgig.geometry.exterior.xy
                elif rgig.geometry.type == 'MultiPolygon':
                    # buffer is necessary as some multi-polygons are self-intersecting
                    allparts = [p.buffer(0) for p in rgig.geometry] 
                    rgig.geometry = shapely.ops.cascaded_union(allparts)
                    if rgig.geometry.type == 'MultiPolygon':
                        print 'ERROR: Polygon %s, %s still a Multipolygon' % (rgig.RGIId, rgig.Name)
                        continue
                    rgi_x, rgi_y = rgig.geometry.exterior.xy
                    #rgig.geometry = rgig.geometry.convex_hull
                    #rgi_x, rgi_y = rgig.geometry.exterior.xy

                #  print centroid of matching glacier
                if i == 0:
                    local_map.set_geometry(shpg.Point(rgig.CenLon, rgig.CenLat), edgecolor='k', marker='x', linewidth=4, 
                                           markersize=100, zorder=50, text='RGI', text_delta=(0.01, 0.0))

                # RGI polygon label
                if not rgig.Name or len(rgig.Name)==1:  # sometimes there is only one strange letter given
                    plabel =  str(rgig.RGIId)+'\n'+str(rgig.Area)+'km2'
                else:
                    #try:
                    plabel =  str(rgig.RGIId)+'\n'+str(rgig.Area)+'km2\n'+rgig.Name.encode('ascii', 'ignore')
                    #except UnicodeEncodeError:
                    #    plabel =  str(rgig.RGIId)+'\n'+str(rgig.Area)+'km2\n'+rgig.Name.encode('ascii', 'ignore')

                local_map.set_geometry(rgig.geometry.exterior, color=colors[i], linewidth=3, label=plabel)
                try:
                    local_map.set_geometry(shpg.Point(lec[lec.index.isin(lin)].lon.values[i], 
                                                      lec[lec.index.isin(lin)].lat.values[i]), 
                                           color=colors[i], marker='x', linewidth=4, markersize=100, zorder=50-i, 
                                           text='LEC:'+str(lec[lec.index.isin(lin)].name.values[i])+
                                           str(lec[lec.index.isin(lin)].ID.values[i]), text_delta=(0.01, 0.01))
                except IndexError:  # when no equivalent equists or <5 equivalents on grid
                    pass
                try:
                    local_map.set_geometry(shpg.Point(gla[gla.index.isin(gin)].LON.values[i], 
                                                      gla[gla.index.isin(gin)].LAT.values[i]), 
                                           color=colors[i], marker='x', linewidth=4, markersize=100, zorder=50-i,
                                           text='GLA:'+str(gla[gla.index.isin(gin)].GLACIER_NAME.values[i]).encode('ascii', 'ignore')+
                                       str(gla[gla.index.isin(gin)].GlaThiDa_ID.values[i]).encode('ascii', 'ignore'), text_delta=(0.01, -0.01))
                except IndexError:  # when no equivalent equists or <5 equivalents on grid
                    pass
                except UnicodeEncodeError:
                    print 'UniCodeEncodeError'
                    pass


            local_map.set_geometry(shpg.Point(rgig.CenLon, rgig.CenLat), c='k', marker='x', markersize=30, zorder=51) 
            # Plot the WGMS point
            local_map.set_geometry(shpg.Point(lon, lat), color='g', marker='x', linewidth=4, markersize=100, zorder=50, 
                                   text='WGMS', text_delta=(0.01, -0.02))

            local_map.set_rgb((local.get_vardata())[..., 0:3])
            local_map.visualize(ax=ax1, addcbar=False)

            local = datasets.GoogleVisibleMap(x=mmlon, y=mmlat, maptype='terrain')
            local_map.set_rgb((local.get_vardata())[..., 0:3])
            local_map.visualize(ax=ax2, addcbar=False)
            plt.subplots_adjust(left=0.04, right=0.80, top=0.94, bottom=0.07)
            plt.legend(bbox_to_anchor=(1.02, 1.), fontsize=18, loc=2, borderaxespad=0, frameon=False, numpoints=1, scatterpoints=1)
            
            pdf.savefig(fig)
            plt.close()
            
            # write results in one step
            results.RGI_ID[results.WGMS_ID == gid] = sortrgi.iloc[0].RGIId
            gla['DIST'] = haversine(lon, lat, gla.LON.values, gla.LAT.values)
            sortgla = gla.sort(columns='DIST')
            results.GlaThiDa_ID[results.WGMS_ID == gid] = sortgla.iloc[0].GlaThiDa_ID
            lec['DIST'] = haversine(lon, lat, lec.lon.values, lec.lat.values)
            sortlec = lec.sort(columns='DIST')
            results.Leclercq_ID[results.WGMS_ID == gid] = sortlec.iloc[0].ID
            

            if curr % 5 == 0:
                print "%s / %s plots done." % (curr, total)
            curr += 1

In [ ]:
results.to_csv(os.path.join(output_dir, 'Automated_links_WGMS_to_RGI_GlaThiDa_Leclercq.csv'))